In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torchvision.utils import save_image

In [2]:
# Definee hyperparameters
image_size = 784
h_dim = 400
latent_dim = 20
batch_size = 128
num_epochs = 10

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='./data/',
                                             train=True,
                                                transform=transforms.ToTensor(),
                                                    download=True)

test_dataset = torchvision.datasets.MNIST(root='./data/',
                                            train=False,
                                                transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                                  batch_size=batch_size,
                                                        shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                                    batch_size=batch_size,
                                                            shuffle=False)

# Create a directory to savee the reconstructed and sampled images (if not exists)
sample_dir = 'samples'
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)


100.0%


Extracting ./data/MNIST\raw\train-images-idx3-ubyte.gz to ./data/MNIST\raw



100.0%
2.0%

Extracting ./data/MNIST\raw\train-labels-idx1-ubyte.gz to ./data/MNIST\raw



100.0%
100.0%


Extracting ./data/MNIST\raw\t10k-images-idx3-ubyte.gz to ./data/MNIST\raw

Extracting ./data/MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data/MNIST\raw



![vae](https://user-images.githubusercontent.com/30661597/78418103-a2047200-766b-11ea-8205-c7e5712715f4.png)

In [4]:
# Define the model
# VAE model
class VAE(nn.Module):
    def __init__(self, image_size=784, h_dim=400, latent_dim=20):
        super(VAE, self).__init__()
        self.fc1 = nn.Linear(image_size, h_dim)
        self.fc2_mean = nn.Linear(h_dim, latent_dim) # mu
        self.fc2_logvar = nn.Linear(h_dim, latent_dim) # logvar
        
        self.fc3 = nn.Linear(latent_dim, h_dim)
        self.fc4 = nn.Linear(h_dim, image_size)
        
    def encode(self, x):
        h = F.relu(self.fc1(x))
        return self.fc2_mean(h), self.fc2_logvar(h)
    
    def reparameterize(self, mu, logvar):
        std = torch.exp(logvar/2) # standard deviation
        eps = torch.randn_like(std) # epsilon
        return mu + eps*std
    
    def decode(self, z):
        h = F.relu(self.fc3(z))
        return F.sigmoid(self.fc4(h))
    
    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar
    
# Define model and optimizer
model = VAE().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)